In [4]:
import numpy as np
#import scipy.stats
#import seaborn as sns
import pandas as pd
import librosa
import librosa.display
from IPython.display import Audio, display
#import scikit_posthocs as sp
#import matplotlib.pyplot as plt
#from matplotlib import gridspec
#from matplotlib.transforms import BlendedGenericTransform
#import scikit_posthocs as sp
#from urllib.request import urlopen
#import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

import os

import keras
import keras.layers as layers
from keras import backend as K
import numpy as np

Using TensorFlow backend.


TODO
* Check alignment between center of window and targets
* Consider edge behavior in terms of data windows (zeroes?)



In [16]:
# some test data to hack around with
test_file = "Assets\\DataShareArchive\\Test\\Noisy\\p232_010.wav"
wav, rate = librosa.core.load(test_file)
fft = librosa.stft(wav)

In [188]:
# Constants and settings
WINDOW_SIZE = 10
FFT_BINS = 1025

In [282]:
# Creating data from clip wave file
def clip_frames(file):
    wav, rate = librosa.core.load(file)
    half_win = WINDOW_SIZE//2
    fft = librosa.stft(wav) # organized as bins, frames
    frames = np.empty(shape=(0,FFT_BINS,WINDOW_SIZE))
    for i in range(0, fft.shape[1] - WINDOW_SIZE):
        frames = np.append(frames,[fft[:,i:i+WINDOW_SIZE]], axis=0)
    return frames


def clip_targets(file):
    wav, rate = librosa.core.load(file)
    half_win = WINDOW_SIZE//2
    fft = librosa.stft(wav)
    targets = np.empty(shape=(0,FFT_BINS))
    for i in range(half_win, fft.shape[1] - half_win):
        targets = np.append(targets, [fft[:,i+half_win:i+half_win+1].flatten()], axis=0)
    return targets

In [286]:
ft = clip_frames(test_file)
tt = clip_targets(test_file)
print(ft.shape, tt.shape)

(110, 1025, 10) (110, 1025)


In [287]:
# Iterate over clean & noisy folders to create frames and targets
def create_data(root):
    clean_dir = root + "\\Clean\\"
    noisy_dir = root + "\\Noisy\\"
    frames = np.empty(shape=(0,FFT_BINS,WINDOW_SIZE))
    targets = np.empty(shape=(0,FFT_BINS))
    for file in os.listdir(clean_dir)[0:2]:
        filename = os.fsdecode(file)
        frames = np.concatenate((frames,clip_frames(noisy_dir + file)), axis=0)
        targets = np.concatenate((targets,clip_targets(clean_dir + file)), axis=0)
    return frames, targets


In [288]:
frames, targets = create_data("Assets\\DataShareArchive\\Test")

In [289]:
targets.shape

(172, 1025)

In [294]:
# Define network

model = keras.Sequential(
     [layers.Dense(FFT_BINS, activation='relu', input_shape=[FFT_BINS]),
      layers.Dense(50, activation='relu'),
      layers.Dense(50, activation='relu'),
     layers.Dense(FFT_BINS, activation='linear')]
)

model.compile(optimizer='rmsprop', loss='mse')

In [296]:
model.fit(targets,targets, epochs=100, batch_size=50)

Epoch 1/100
172/172 [==============================] - 0s 388us/step - loss: 3.1067
Epoch 2/100
172/172 [==============================] - 0s 319us/step - loss: 3.0355
Epoch 3/100
172/172 [==============================] - 0s 290us/step - loss: 2.9330
Epoch 4/100
172/172 [==============================] - 0s 273us/step - loss: 2.7897
Epoch 5/100
172/172 [==============================] - 0s 284us/step - loss: 2.6458
Epoch 6/100
172/172 [==============================] - 0s 278us/step - loss: 2.5152
Epoch 7/100
172/172 [==============================] - 0s 276us/step - loss: 2.3899
Epoch 8/100
172/172 [==============================] - 0s 278us/step - loss: 2.3195
Epoch 9/100
172/172 [==============================] - 0s 288us/step - loss: 2.2331
Epoch 10/100
172/172 [==============================] - 0s 284us/step - loss: 2.1631
Epoch 11/100
172/172 [==============================] - 0s 290us/step - loss: 2.1247
Epoch 12/100
172/172 [==============================] - 0s 284us/step - lo

172/172 [==============================] - 0s 670us/step - loss: 0.6544
Epoch 98/100
172/172 [==============================] - 0s 325us/step - loss: 0.6905
Epoch 99/100
172/172 [==============================] - 0s 267us/step - loss: 0.6711
Epoch 100/100
172/172 [==============================] - 0s 255us/step - loss: 0.6608


In [70]:
print(frames[0][5][0])

print(targets[0][0][0])

(-0.04013502-5.551115e-17j)
(0.16505659+1.3877788e-17j)


In [214]:
fft.shape
fft[:,1:2].shape

(1025, 1)

In [66]:
os.listdir("Assets\\DataShareArchive\\Test\\Noisy\\")[0:5]

['p232_001.wav',
 'p232_002.wav',
 'p232_003.wav',
 'p232_005.wav',
 'p232_006.wav']